# TP d'analyse de la météo

Sur le site weatherunderground, nous avons récupéré la météo pour les éléments suivants:

    villes = ['Paris', 'Marseille', 'Grenoble', 'Lille', 'Strasbourg', 'Nantes', 'Bordeaux']
    year   = '2017'
    month  = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    day    = ['01', '08', '15', '22']

[Pour info] Le script de constitution du corpus est récupérable à l'adresse : http://www-connex.lip6.fr/~guigue/weatherdata.py <br>
Pour le TP, les données sont à télécharger à l'adresse : http://www-connex.lip6.fr/~guigue/weatherJSON.pkl

In [1]:
# definition des import nécessaires
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl

## Chargement des données + affichage basique
Code fourni

1. Chargement des données pickle
1. Exploration des données disponibles par affichages multiples

In [3]:
# chemin vers le .pkl que vous aurez préalablement téléchargé:
path2file = "ressources/weatherJSON.pkl"

data = pkl.load(open(path2file, "rb"))

In [4]:
# analyse basique:
print(len(data)) # 336 = nb_villes x nb_mois x nb_jours
print(len(data[0])) # Chaque entrée est composée de deux choses
print(data[0].keys())
print(data[0]['response'].keys()) # listes des clés du premier dico associé à chaque entrée
# => 'response' n'a pas l'air intéressante

print(data[0]['history'].keys()) # listes des clés du deuxième dico associé à chaque entrée
# => Beaucoup mieux... On poursuit l'exploration

336
2
dict_keys(['response', 'history'])
dict_keys(['version', 'termsofService', 'features'])
dict_keys(['date', 'utcdate', 'observations', 'dailysummary'])


In [5]:
# exploration d'une entrée:
print(data[0]['history']['dailysummary'][0].keys())
print(data[0]['history']['dailysummary'][0]) # nous allons nous concentrer sur les résumés quotidiens
# => les infos dans 'observations' sont plus précises... Mais on les laisse de coté

dict_keys(['date', 'fog', 'rain', 'snow', 'snowfallm', 'snowfalli', 'monthtodatesnowfallm', 'monthtodatesnowfalli', 'since1julsnowfallm', 'since1julsnowfalli', 'snowdepthm', 'snowdepthi', 'hail', 'thunder', 'tornado', 'meantempm', 'meantempi', 'meandewptm', 'meandewpti', 'meanpressurem', 'meanpressurei', 'meanwindspdm', 'meanwindspdi', 'meanwdire', 'meanwdird', 'meanvism', 'meanvisi', 'humidity', 'maxtempm', 'maxtempi', 'mintempm', 'mintempi', 'maxhumidity', 'minhumidity', 'maxdewptm', 'maxdewpti', 'mindewptm', 'mindewpti', 'maxpressurem', 'maxpressurei', 'minpressurem', 'minpressurei', 'maxwspdm', 'maxwspdi', 'minwspdm', 'minwspdi', 'maxvism', 'maxvisi', 'minvism', 'minvisi', 'gdegreedays', 'heatingdegreedays', 'coolingdegreedays', 'precipm', 'precipi', 'precipsource', 'heatingdegreedaysnormal', 'monthtodateheatingdegreedays', 'monthtodateheatingdegreedaysnormal', 'since1sepheatingdegreedays', 'since1sepheatingdegreedaysnormal', 'since1julheatingdegreedays', 'since1julheatingdegreeday

## Les vérrous à lever pour pouvoir traiter les données:

### Q1. On remarque qu'il y a des dictionnaires dans des dictionaires...
Il faut tout remettre à plat pour une importation facile.<br>
La méthode pd.io.json.json_normalize fait ça ! 

    df = pd.io.json.json_normalize(data[0]['history']['dailysummary'][0])

### Q2. On a une liste de JSON (liée à la manière dont les données ont été collectées)
=> On veut remettre tout ça dans un tableau <br>
Il faut parcourir tous les i entre 0 et 335 :
    
    data[i]['history']['dailysummary'][0]
    
Et ajouter des lignes selon la procédure:

    df = df.append( DataFrame object)

### Q3. L'information sur la ville a été malencontreusement égarée... Il faut la remettre dans la table
Etant donné les villes dans l'ordre :
    
    villes = ['Paris', 'Marseille', 'Grenoble', 'Lille', 'Strasbourg', 'Nantes', 'Bordeaux']
    coords = [[48.853340,2.348342], [43.295421,5.374743],[45.185850, 5.736848],\
                  [50.636907,3.068034], [48.580519, 7.751759], [47.215708, -1.553418], [44.841386,-0.570341]]

Les 48 premières entrées correspondent à Paris, les 48 suivantes à Marseille etc...
En pratique, vous ajouterez 4 colonnes:
1. colonne 'ville' contenant la chaine décrivant la ville
1. colonne 'indexville' contenant un entier de 0 à 6 correspondant à la ville
1. colonne 'latitude' contenant la première coordonnée

    Ces informations sont dupliquées pour toutes les lignes... On ne s'embete pas avec des jointures
1. colonne 'longitude' contenant la seconde coordonnée

Implémentation: la génération de ces colonnes est un bon exercice de révision des boucles.

Rappel:
* Créer une liste contenant N fois une valeur v:
    
     li = [v]*N
     
* Concaténer deux listes

    li = li1 + li2

In [ ]:
# mise en forme des données (remise à plat des dicos dans les dicos)
df = pd.io.json.json_normalize(data[0]['history']['dailysummary'][0])

### Q4. Affichage des infromations récupérées sur une carte

1. Afficher les villes sous la forme de points en 2D (scatter plot sur les 7 villes)
1. Ajouter les noms des villes

       plt.text(x, y, montexte)
1. Calculer le nombre total de jours de précipitation sur les 48 jours disponibles pour chaque ville, ainsi que le total de pluie en mm

    C'est à vous de trouver la bonne colonne à exploiter
    ATTENTION: La colonne est définie en chaine de caractères...
    Pour faire des opérations, il faudra convertir en utilisant la syntaxe suivante:
    
        np.array(df['NAME_COLONNE'], dtype=float)
    
    Afin de gagner du temps dans la suite, il est intéressant de modifier la défintion de la colonne:
    
        df['NAME_COLONNE'] = np.array(df['NAME_COLONNE'], dtype=float)
        df['NAME_COLONNE'] = np.array(df['NAME_COLONNE'], dtype=int) # variables entières comme date.mon

    Utiliser cette syntaxe sur la pluie, la neige, les mois (date.mon), le brouillard... Toutes les variables susceptibes de vous intéresser
1. Extrapoler ces chiffres sur l'année pour chaque ville (règle de proportionnalité)

In [ ]:
# Diviser le code de la réponse pour que ce soit plus simple

### Q5. Etude des corrélations
Pour les deux questions suivantes, vous utiliserez une boucle for sur une liste contenant les noms de variables afin de rendre le code plus élégant. Pour le coefficient de corrélation, vous utiliserez directement:
    
    np.corrcoef(X,Y)[0,1]

1. Comparons les coefficients de corrélation entre la variable 'pluie' et les variables 'mois', 'latitude' et 'longitude'

1. Comparons ensuite la variable donnant les mm de pluie par rapport aux variables de références

In [4]:
import numpy as np
X = np.random.rand(100, 3)

print(np.corrcoef(X.T))

[[ 1.         -0.01290968  0.12103144]
 [-0.01290968  1.         -0.04056042]
 [ 0.12103144 -0.04056042  1.        ]]


### Q6. Etude de la distribution jointe P('precipm', 'date.mon')
Nous distinguerons 10 niveaux de précipitation. On ne discrétisera pas les mois (qui le sont déjà)

1. Afficher un scatter plot des deux variables

1. Calculer la loi jointe et l'afficher avec imshow

    ATTENTION aux indices... Les mois vont de 1 à 12, les matrices de 0 à 11.

1. Calculer la distribution conditionnelle p('Precipm' | 'date.mon') et l'afficher

    Note: cette distribution conditionnelle n'apporte pas vraiment d'information... C'est juste un exercice scolaire

In [ ]:
# affichage

In [ ]:
# calcul de la loi jointe

In [ ]:
# calcul de la loi conditionnelle

### Q7. Proposer une étude amusante ou une illustration originale sur ces données